In [310]:
import pandas as pd
from datetime import datetime,date
import re
import os

In [311]:

# Paths
name_transit_time  = "../Input_fijo/2019-2022 DMS1__Transit Time.xlsx"
name_transit_time_23 = "../input_bauti/ultima_campaña/2022-2023 DMS1_Transit Times.xlsx"

# Output
path_output = '../input_bauti/historico/'
name_output = 'df_historico.xlsx'

In [312]:
df_consolidado = pd.read_excel(name_transit_time,
                                sheet_name="Consolidado",
                                header=0,
                                #dtype = {"tiempo_real":datetime},
                                names = ['id_dms',
                                            'año',
                                            'zone_x',
                                            'subZone',
                                            'batch',
                                            'truck_num',
                                            'truck Type',
                                            'delivery_num',
                                            'start_dt',
                                            'end_dt',
                                            'tiempo_min_zone',
                                            'tiempo_max_zone',
                                            'temp_min',
                                            'temp_max',
                                            'obs_temp',
                                            'tiempo_real',
                                            'obs_tiempo',
                                            'obs_peso',
                                            'kg',
                                            'establecimiento',
                                            'tiempo',
                                            'conteo_batch',
                                            'conteo_subzone',
                                            'logueo_manual',
                                        ]                                                                   
                                )
df_consolidado_2023 = pd.read_excel(name_transit_time_23,
                            sheet_name="Sheet1",
                            header=0,
                            #dtype = {"tiempo_real":datetime},
                            names= [
                                    'año',
                                    'zone_x',
                                    'subZone',
                                    'batch',
                                    'truck_num',
                                    'truck Type',
                                    'delivery_num',
                                    'start_dt',
                                    'end_dt',
                                    'tiempo_min_zone',
                                    'tiempo_max_zone',
                                    'temp_min',
                                    'temp_max',
                                    'obs_temp',
                                    'tiempo_real',
                                    'obs_tiempo',
                                    'obs_peso',
                                    'kg',
                                    'establecimiento'
                            ]
                        )

### Concatenacion de nuevos registros.

In [313]:
# Elimino registros del 2022 - 2023 que no cuentan con zona o subzona asignada.
print('shape antes de elminar = %s Filas - %s Columnas' %(df_consolidado_2023.shape[0], df_consolidado_2023.shape[1]))

df_consolidado_2023.dropna(axis=0, how='any', subset=['zone_x', 'subZone', 'tiempo_real'], inplace=True)

print('shape luego de elminar = %s Filas - %s Columnas' %(df_consolidado_2023.shape[0], df_consolidado_2023.shape[1]))

shape antes de elminar = 23943 Filas - 19 Columnas
shape luego de elminar = 23101 Filas - 19 Columnas


In [314]:
# Me quedo con las columnas en comun.
cols_considerar = df_consolidado_2023.columns.to_list()
df_consolidado = df_consolidado[cols_considerar]
df_consolidado.shape

(16518, 19)

In [315]:
# Funcion para formatear los datos, y generar una columna nueva, que tenga en cuenta el año y la fecha de entrega.
def convert_to_datetime (row, date_row):
    year = row['año']
    date_time_str = row[date_row]
    
    return datetime.strptime(f"{year}-{date_time_str}", "%Y-%d/%m . %H:%M")


df_consolidado['start_date'] = df_consolidado.apply(lambda x: convert_to_datetime(x, 'start_dt'), axis=1).dt.strftime('%Y-%m-%d %H:%M:%S')
df_consolidado['end_date'] = df_consolidado.apply(lambda x: convert_to_datetime(x, 'end_dt'), axis=1).dt.strftime('%Y-%m-%d %H:%M:%S')
df_consolidado_2023['start_date'] = df_consolidado_2023.apply(lambda x: convert_to_datetime(x, 'start_dt'), axis=1).dt.strftime('%Y-%m-%d %H:%M:%S')
df_consolidado_2023['end_date'] = df_consolidado_2023.apply(lambda x: convert_to_datetime(x, 'end_dt'), axis=1).dt.strftime('%Y-%m-%d %H:%M:%S')

In [316]:
# Obtengo los registros que ya estaban cargados en el input inicial.
list_start_date = df_consolidado['start_date'].unique().tolist()
list_end_date = df_consolidado['end_date'].unique().tolist()

# Filtro las entregas que ya se realizaron
cols_antes = df_consolidado_2023.shape[0]

mask = ((df_consolidado_2023['start_date'].isin(list_start_date)) & (df_consolidado_2023['end_date'].isin(list_start_date)))
df_consolidado_2023 = df_consolidado_2023[~mask]

# Dropeo las columnas creadas ya que no seran de utilidad.
df_consolidado_2023.drop(columns=['start_date', 'end_date'], axis=1, inplace=True)
df_consolidado.drop(columns=['start_date', 'end_date'], axis=1, inplace=True)

print('Registros a agregar - %s Filas (de %s)' %(df_consolidado_2023.shape[0], cols_antes))

Registros a agregar - 22285 Filas (de 23101)


In [317]:
# Agrego informacion faltante.
df_consolidado = pd.concat([df_consolidado, df_consolidado_2023], ignore_index=False)
df_consolidado.drop_duplicates(inplace=True)
df_consolidado.sort_values(['año', 'start_dt'], inplace=True, ignore_index=True)

# Unifico los nombres de los establecimientos.
df_consolidado['establecimiento'] = df_consolidado['establecimiento'].str.lower()

print('Shape = (%s, %s)' %(df_consolidado.shape[0], df_consolidado.shape[1]))
df_consolidado.head(3)

Shape = (38768, 19)


,año,zone_x,subZone,batch,truck_num,truck Type,delivery_num,start_dt,end_dt,tiempo_min_zone,tiempo_max_zone,temp_min,temp_max,obs_temp,tiempo_real,obs_tiempo,obs_peso,kg,establecimiento
0,2020,F,F3,UF20610N,130915,CH,NaN,01/02 . 12:02,01/02 . 20:19,04:00,05:30,27.3,38.58,OK,01:30:00,Early,OK,22840.0,la tijereta
1,2020,F,F3,UF20610N,130915,CH,NaN,01/02 . 12:02,01/02 . 20:19,04:00,05:30,27.3,38.58,OK,01:30:00,Early,OK,22840.0,la tijereta
2,2020,F,F3,UF20610N,130952,CH,NaN,01/02 . 13:57,01/02 . 20:44,04:00,05:30,27.6,37.40,OK,03:35:00,Early,OK,23420.0,la tijereta


In [318]:
# La columna tiempo real tiene que tener los valores en estilo H:M:S, pero algunos
# valores vienen en formato '1900-01-01  H:M:S', voy a corregirlo con regex.
# Esto se debe a que cuando una hora supera las 24 hs se le agrega la fecha esa.
def format_time(value):
    value_datetime = pd.to_datetime(value)
    
    hora_mas_24 = int(value_datetime.hour)+24
    minutos = f'{value_datetime.minute:02d}'
    segundos =  f'{value_datetime.second:02d}'
    
    return f'{hora_mas_24}:{minutos}:{segundos}'

In [319]:
# Transformo la columna a str, para trabajar con texto.
df_consolidado['tiempo_real'] = df_consolidado['tiempo_real'].astype(str)

# Obtengo los registros que superan las 24 hs.
mask_reg_mas_24hs = df_consolidado['tiempo_real'].str.contains('1900-01-01')


df_consolidado.loc[mask_reg_mas_24hs,'tiempo_real'] = df_consolidado.loc[mask_reg_mas_24hs,'tiempo_real'].apply(format_time)

# Dropeo horarios vacios.
df_consolidado.dropna(subset='tiempo_real', axis=0, how='any', inplace=True)

In [320]:
# Creo una nueva carpeta para este archivo historico y guardo el archivo.
if not os.path.exists(path_output):
    os.makedirs(path_output)

df_consolidado.to_excel(path_output+name_output, index=False)
